In [ ]:
#importing the required libraries
from simpletransformers.ner import NERModel
import pandas as pd
import logging
import re
from nltk.tokenize import TreebankWordTokenizer, sent_tokenize
from transformers import pipeline
from transformers import AutoTokenizer, AutoModelForTokenClassification
import openpyxl
from openpyxl import load_workbook
logging.basicConfig(level=logging.DEBUG)
transformers_logger = logging.getLogger('transformers')
transformers_logger.setLevel(logging.WARNING)
import datasets
from datasets import Dataset
import re
from datasets import Features, Value, ClassLabel
from datasets import load_dataset
from datasets import concatenate_datasets, load_dataset
import os

#importing the conll file(test or train or validate)
input_connl = '/host/Sahithi/myenv/an_validate_total.conll'

#creating the dataset from the total annotated file
def create_dataset_from_annotated_file(i):
    input_file = i
    sentences = []
    current_sentence = []
        with open(input_file, "r") as f:
        for line in f:
            line = line.strip()
            if line :
                if line.startswith('.'):
                    word, _, _, label = line.split()
                    current_sentence.append((word, label))
                    if current_sentence:
                        sentences.append(current_sentence)
                    current_sentence = []
                else:
                    word, _, _, label = line.split()
                    current_sentence.append((word, label))
            else:

                if current_sentence:
                    sentences.append(current_sentence
                current_sentence = []
    if current_sentence:
        sentences.append(current_sentence)
    
    c=0
    conll_data = {

        "tokens": [],
        "ner_tags": []
    }
    s=0
    for sent_id, sentence in enumerate(sentences):
        tokens, labels = zip(*sentence)
        s+=1
        c+=1
        conll_data = {
    
        "tokens": [],
        "ner_tags": []
    }
        conll_data["tokens"] = [list(tokens)]
        conll_data["ner_tags"] = [convert_to_int_labels(list(labels), labels_list)[0]]
        class_names = ['O', 'I-RRMS', 'I-Disease_Subtype', 'B-RRMS', 'I-SPMS', 'B-Disease_Subtype', 'B-EDSS', 'B-SPMS', 'B-Relapse_relative_date', 'I-Relapse_Date', 'I-EDSS', 'B-Relapse_date', 'B-EDSS_Value', 'I-Temporal_history_of_relapse', 'I-PPMS', 'B-Temporal_history_of_relapse', 'I-PRMS', 'B-MS_no_subtype', 'B-EDSS_Date', 'I-EDSS_Value', 'B-Relapse_Date', 'I-MS_no_subtype', 'B-Relapse_Temporality', 'I-Relapse_Temporality', 'B-PPMS', 'I-Relapse_date', 'I-CIS', 'I-Relapse_year', 'I-EDSS_Date', 'I-Relapse_relative_date', 'B-Relapse_year', 'B-CIS', 'B-PRMS', 'B-Severity_Severe', 'B-Relapse_Severity', 'B-Functional_Disability', 'B-Severity_Mild', 'I-Severity_Severe']
        ft = Features({
                       'tokens': datasets.Sequence(feature=Value(dtype='string', id=None), length=-1, id=None),
                       'ner_tags': datasets.Sequence(feature = ClassLabel(names=class_names), length = -1)})
        if c==1:     
            a = Dataset.from_dict(conll_data,features = ft)
        else:
            b = Dataset.from_dict(conll_data,features = ft)
            assert b.features.type == a.features.type
            a = concatenate_datasets([a,b])
    return a
                                     
#conversion of string labels to integers
def convert_to_int_labels(labels,labels_list):
    label_to_int = {label: i for i, label in enumerate(labels_list)}
    converted_ner_tags = []
# Iterate through each example in the dataset and convert NER tags
    converted_tags = [label_to_int[tag] for tag in labels]
    converted_ner_tags.append(converted_tags)
    return converted_ner_tags
 
                                     
#resultant dataset                                     
test = create_dataset_from_annotated_file(input_connl)
 
                                     
#loading the model the model                                     
from transformers import pipeline
model_checkpoint = "alvaroalon2/biobert_diseases_ner"
token_classifier = pipeline(
    "ner", model=model_checkpoint, aggregation_strategy="simple"
)
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)
                                     
                                     
                                    
ner_feature = test.features["ner_tags"]
label_names = ner_feature.feature.names
                                     
                                     
#aligning the labels with the tokens                                   
def align_labels_with_tokens(labels, word_ids):
    new_labels = []
    current_word = None
    for word_id in word_ids:
        if word_id != current_word:
            # Start of a new word!
            current_word = word_id
            label = -100 if word_id is None else labels[word_id]
            new_labels.append(label)
        elif word_id is None:
            # Special token
            new_labels.append(-100)
        else:
            # Same word as previous token
            label = labels[word_id]
            # If the label is B-XXX we change it to I-XXX
            if label % 2 == 1:
                label += 1
            new_labels.append(label)
    return new_labels
                                     
                                     
def tokenize_and_align_labels(examples):
    tokenized_inputs = tokenizer(
        examples["tokens"], truncation=True, is_split_into_words=True
    )
    all_labels = examples["ner_tags"]
    new_labels = []
    for i, labels in enumerate(all_labels):
        word_ids = tokenized_inputs.word_ids(i)
        new_labels.append(align_labels_with_tokens(labels, word_ids))

    tokenized_inputs["labels"] = new_labels
    return tokenized_inputs
                                     

tokenized_datasets = test.map(
    tokenize_and_align_labels,
    batched=True,
    remove_columns=test.column_names,
)
                                     
                                     
# Evaluation of the model over the complete dataset                                   
import torch
for batch in test_dataloader:
    with torch.no_grad():
        outputs = model(**batch)
    predictions = outputs.logits.argmax(dim=-1)
    labels = batch["labels"]
    # Necessary to pad predictions and labels for being gathered
    predictions = accelerator.pad_across_processes(predictions, dim=1, pad_index=-100)
    labels = accelerator.pad_across_processes(labels, dim=1, pad_index=-100)
    predictions_gathered = accelerator.gather(predictions)
    labels_gathered = accelerator.gather(labels)
    true_predictions, true_labels = postprocess(predictions_gathered, labels_gathered)
                                     
    token_names=[]
    true_predictions_extended=[]
    true_labels_extended=[]
                                     
    for ids in batch['input_ids']:
        tokens=tokenizer.batch_decode(ids,skip_special_tokens=True)
        token_names.extend(tokens)
        for i in range(token_names.count('')):
            token_names.remove('')
    for i in true_predictions:
        true_predictions_extended.extend(i)
    for i in true_labels:
        true_labels_extended.extend(i)                               
    if(len(token_names)==len(true_labels_extended)):
        total=pd.DataFrame({
            "token_name":token_names,
            "true_label":true_labels_extended,
            "true_prediction":true_predictions_extended,
        })
    metric.add_batch(predictions=true_predictions, references=true_labels)
                                     
                                     
results = metric.compute()
#printing the results
print(
    f"epoch:",
    {
        key: results[f"overall_{key}"]
        for key in ["precision", "recall", "f1", "accuracy"]
    },
)